In [1]:
## Has a lot of already implemented adversarial attacks like FGSM, PGD, BIM, CW, DeepFool etc.

!pip install torchattacks

     |████████████████████████████████| 105 kB 7.4 MB/s 


In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import time
from datetime import datetime
from sklearn.model_selection import train_test_split
import cv2
import pickle
import torchattacks
import random
from torchvision import models
import copy
from PIL import Image
import io
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# parameters
RANDOM_SEED = 42
BATCH_SIZE = 64

IMG_SIZE = 32
N_CLASSES = 10

In [5]:
transformations = transforms.Compose([transforms.Resize((32, 32)),
                                 transforms.ToTensor(),])

train_dataset = datasets.MNIST(root='mnist_data', 
                               train=True, 
                               transform=transformations,
                               download=True)

valid_dataset = datasets.MNIST(root='mnist_data', 
                               train=False, 
                               transform=transformations)

valid_dataset = list(train_dataset)[int(len(train_dataset)*0.8):]
train_dataset = list(train_dataset)[:int(len(train_dataset)*0.8)]

train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=True)

valid_loader = DataLoader(dataset=valid_dataset, 
                          batch_size=BATCH_SIZE, 
                          shuffle=False)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to mnist_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



In [6]:
## Code loosely from https://pytorch.org/tutorials/beginner/fgsm_tutorial.html

class FGSM():

    def __init__(self, loss_fn, fgsm_epsilon):

        self.fgsm_epsilon = fgsm_epsilon
        self.loss_fn = loss_fn

    def get_sample(self, model, x, y):
    
        x.requires_grad = True
        _, logits = model(x)
        prediction = F.softmax(logits, dim=1)
        loss = self.loss_fn(prediction, y)
        loss.backward()
        
        x_grad = x.grad.data
        sign = x_grad.sign()
        x_adv = x + self.fgsm_epsilon*sign
        x_adv = torch.clamp(x_adv, 0, 1)

        return x_adv

In [7]:
## Code from https://towardsdatascience.com/know-your-enemy-7f7c5038bdf3

class PGD():

    def __init__(self, loss_fn, num_steps, step_size, eps, step_norm, eps_norm, clamp=(0,1)):

        self.loss_fn = loss_fn
        self.num_steps = num_steps
        self.step_size = step_size
        self.eps = eps
        self.step_norm = step_norm
        self.eps_norm = eps_norm
        self.clamp = clamp

    def get_sample(self, model, x, y, y_target=None):

        x_adv = x.clone().to(device)
        targeted = y_target is not None
        num_channels = x.shape[1]

        for i in range(self.num_steps):
            # _x_adv = x_adv.clone().detach().requires_grad_(True)
            _x_adv = x_adv.clone()
            _x_adv.requires_grad = True

            _, logits = model(_x_adv)
            prediction = F.softmax(logits, dim=1)
            loss = self.loss_fn(prediction, y_target if targeted else y)
            loss.backward()

                # Force the gradient step to be a fixed size in a certain norm
            if self.step_norm == 'inf':
                gradients = _x_adv.grad.sign() * self.step_size
            else:
                    # Note .view() assumes batched image data as 4D tensor
                gradients = _x_adv.grad * self.step_size / _x_adv.grad.view(_x_adv.shape[0], -1)\
                    .norm(self.step_norm, dim=-1)\
                    .view(-1, num_channels, 1, 1)

            if targeted:
                    # Targeted: Gradient descent with on the loss of the (incorrect) target label
                    # w.r.t. the image data
                x_adv -= gradients
            else:
                    # Untargeted: Gradient ascent on the loss of the correct label w.r.t.
                    # the model parameters
                x_adv += gradients

            # Project back into l_norm ball and correct range
            if self.eps_norm == 'inf':
                # Workaround as PyTorch doesn't have elementwise clip
                x_adv = torch.max(torch.min(x_adv, x + self.eps), x - self.eps)
            else:
                delta = x_adv - x

                # Assume x and x_adv are batched tensors where the first dimension is
                # a batch dimension
                mask = delta.view(delta.shape[0], -1).norm(self.eps_norm, dim=1) <= self.eps

                scaling_factor = delta.view(delta.shape[0], -1).norm(self.eps_norm, dim=1)
                scaling_factor[mask] = self.eps

                # .view() assumes batched images as a 4D Tensor
                delta *= self.eps / scaling_factor.view(-1, 1, 1, 1)

                x_adv = x + delta
                
            x_adv = x_adv.clamp(*self.clamp)

        # return x_adv.detach().cpu()
        return x_adv

In [8]:
class ClassificationDefense(nn.Module):

    def __init__(self, n_adv_classes, n_classes):
        super(ClassificationDefense, self).__init__()
        
        self.feature_extractor = nn.Sequential(            
            nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1),
            nn.Tanh(),
            nn.AvgPool2d(kernel_size=2),
            nn.Conv2d(in_channels=16, out_channels=120, kernel_size=5, stride=1),
            nn.Tanh()
        )

        self.adv_classifier = nn.Sequential(
            nn.Linear(in_features=120, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_adv_classes),
        )
        
        self.class_classifier = nn.Sequential(
            nn.Linear(in_features=120 + n_adv_classes, out_features=84),
            nn.Tanh(),
            nn.Linear(in_features=84, out_features=n_classes),
        )

    def forward(self, x):
        x = self.feature_extractor(x)
        x = torch.flatten(x, 1)
        adv_logits = self.adv_classifier(x)
        comb_features = torch.cat((x, adv_logits), dim=1)
        logits = self.class_classifier(comb_features)
        # probs = F.softmax(logits, dim=1)
        # return logits, probs
        return adv_logits, logits


In [9]:
def train(train_loader, model, criterion, optimizer, device):
    '''
    Function for the training step of the training loop
    '''

    correct_class_pred = 0
    correct_adv_pred = 0 
    n = 0
    
    model.train()
    running_class_loss = 0
    running_adv_loss = 0
    
    for X, y_true in tqdm(train_loader):

        optimizer.zero_grad()
        
        X = X.to(device)
        y_true = y_true.to(device)

        y_adv = torch.zeros(y_true.size(), dtype=torch.long, device=device)
    
        # Forward pass
        adv_hat, class_hat = model(X) 
        adv_loss = criterion(adv_hat, y_adv)
        class_loss = criterion(class_hat, y_true)

        total_loss = class_loss + adv_loss
        
        running_class_loss += class_loss.item() * X.size(0)
        running_adv_loss += adv_loss.item() * X.size(0)

        # Backward pass
        total_loss.backward()
        optimizer.step()

        class_prob = F.softmax(class_hat, dim=1)
        _, predicted_class_labels = torch.max(class_prob, 1)

        adv_prob = F.softmax(adv_hat, dim=1)
        _, predicted_adv_labels = torch.max(adv_prob, 1)

        n += y_true.size(0)
        correct_class_pred += (predicted_class_labels == y_true).sum()
        correct_adv_pred += (predicted_adv_labels == y_adv).sum()
        
    epoch_class_loss = running_class_loss / len(train_loader.dataset)
    epoch_adv_loss = running_adv_loss / len(train_loader.dataset)
    class_acc = correct_class_pred.float() / n
    adv_acc = correct_adv_pred.float() / n
    return model, optimizer, epoch_class_loss, epoch_adv_loss, class_acc, adv_acc

In [10]:
def adv_train(train_loader, model, criterion, optimizer, device, adv_training_prob):
    '''
    Function for the training step of the training loop
    '''

    correct_class_pred = 0
    correct_adv_pred = 0 
    n = 0
    
    model.train()
    running_class_loss = 0
    running_adv_loss = 0
    
    for X, y_true in tqdm(train_loader):

        rand = random.random()
        
        optimizer.zero_grad()
        
        X = X.to(device)
        y_true = y_true.to(device)
        y_adv = torch.zeros(y_true.size(), dtype=torch.long, device=device)
        
        if adv_training_prob > rand:
            
            attack = 'pgd-linf'

            if attack == 'pgd-linf':
                y_adv = torch.ones(y_true.size(), dtype=torch.long, device=device)
                x_adv = pgd_linf.get_sample(model, X, y_true)
            elif attack == 'pgd-l2':
                y_adv = torch.ones(y_true.size(), dtype=torch.long, device=device) * 2
                x_adv = pgd_l2.get_sample(model, X, y_true)
            elif attack == 'fgsm':
                y_adv = torch.ones(y_true.size(), dtype=torch.long, device=device)
                x_adv = fgsm.get_sample(model, X, y_true)
            elif attack == 'cw':
                y_adv = torch.ones(y_true.size(), dtype=torch.long, device=device) * 2
                x_adv = cw(X, y_true)
            elif attack == 'deepfool':
                y_adv = torch.ones(y_true.size(), dtype=torch.long, device=device) * 2
                x_adv = deepfool(X, y_true)
        
            # Forward pass
            adv_hat, class_hat = model(x_adv)
        else:
            adv_hat, class_hat = model(X)

        adv_loss = criterion(adv_hat, y_adv)
        class_loss = criterion(class_hat, y_true)

        total_loss = class_loss + adv_loss
        
        running_class_loss += class_loss.item() * X.size(0)
        running_adv_loss += adv_loss.item() * X.size(0)

        # Backward pass
        total_loss.backward()
        optimizer.step()

        class_prob = F.softmax(class_hat, dim=1)
        _, predicted_class_labels = torch.max(class_prob, 1)

        adv_prob = F.softmax(adv_hat, dim=1)
        _, predicted_adv_labels = torch.max(adv_prob, 1)

        n += y_true.size(0)
        correct_class_pred += (predicted_class_labels == y_true).sum()
        correct_adv_pred += (predicted_adv_labels == y_adv).sum()
        
    epoch_class_loss = running_class_loss / len(train_loader.dataset)
    epoch_adv_loss = running_adv_loss / len(train_loader.dataset)
    class_acc = correct_class_pred.float() / n
    adv_acc = correct_adv_pred.float() / n
    return model, optimizer, epoch_class_loss, epoch_adv_loss, class_acc, adv_acc

In [11]:
def validate(valid_loader, model, criterion, device):
    '''
    Function for the validation step of the training loop
    '''
   
    correct_class_pred = 0 
    correct_adv_pred = 0
    n = 0
    
    model.eval()
    running_class_loss = 0
    running_adv_loss = 0
    
    for X, y_true in tqdm(valid_loader):
    
        X = X.to(device)
        y_true = y_true.to(device)
        y_adv = torch.zeros(y_true.size(), dtype=torch.long, device=device)

        # Forward pass and record loss
        adv_hat, class_hat = model(X) 
        adv_loss = criterion(adv_hat, y_adv)
        class_loss = criterion(class_hat, y_true)

        total_loss = class_loss + adv_loss
        
        running_class_loss += class_loss.item() * X.size(0)
        running_adv_loss += adv_loss.item() * X.size(0)

        class_prob = F.softmax(class_hat, dim=1)
        _, predicted_class_labels = torch.max(class_prob, 1)

        adv_prob = F.softmax(adv_hat, dim=1)
        _, predicted_adv_labels = torch.max(adv_prob, 1)

        n += y_true.size(0)
        correct_class_pred += (predicted_class_labels == y_true).sum()
        correct_adv_pred += (predicted_adv_labels == y_adv).sum()

    epoch_class_loss = running_class_loss / len(train_loader.dataset)
    epoch_adv_loss = running_adv_loss / len(train_loader.dataset)
    class_acc = correct_class_pred.float() / n
    adv_acc = correct_adv_pred.float() / n
        
    return model, epoch_class_loss, epoch_adv_loss, class_acc, adv_acc

In [12]:
def adv_validate(valid_loader, model, criterion, device,  bb_model=None):
    '''
    Function for the validation step of the training loop
    '''
   
    correct_class_pred = 0 
    correct_adv_pred = 0
    n = 0
    
    model.eval()
    running_class_loss = 0
    running_adv_loss = 0
    
    # # White box attack
    # if not bb_model:
    #     bb_model = model

    for X, y_true in tqdm(valid_loader):
    
        X = X.to(device)
        y_true = y_true.to(device)

        attack = 'pgd-linf'

        if attack == 'pgd-linf':
            y_adv = torch.ones(y_true.size(), dtype=torch.long, device=device)
            x_adv = pgd_linf.get_sample(model, X, y_true)
        elif attack == 'pgd-l2':
            y_adv = torch.ones(y_true.size(), dtype=torch.long, device=device) * 2
            x_adv = pgd_l2.get_sample(model, X, y_true)
        elif attack == 'fgsm':
            y_adv = torch.ones(y_true.size(), dtype=torch.long, device=device)
            x_adv = fgsm.get_sample(model, X, y_true)
        elif attack == 'cw':
            y_adv = torch.ones(y_true.size(), dtype=torch.long, device=device) * 2
            x_adv = cw(X, y_true)
        elif attack == 'deepfool':
            y_adv = torch.ones(y_true.size(), dtype=torch.long, device=device) * 2
            x_adv = deepfool(X, y_true)

        # Forward pass and record loss
        adv_hat, class_hat = model(x_adv)

        adv_loss = criterion(adv_hat, y_adv)
        class_loss = criterion(class_hat, y_true)

        total_loss = class_loss + adv_loss
        
        running_class_loss += class_loss.item() * X.size(0)
        running_adv_loss += adv_loss.item() * X.size(0)

        class_prob = F.softmax(class_hat, dim=1)
        _, predicted_class_labels = torch.max(class_prob, 1)

        adv_prob = F.softmax(adv_hat, dim=1)
        _, predicted_adv_labels = torch.max(adv_prob, 1)

        n += y_true.size(0)
        correct_class_pred += (predicted_class_labels == y_true).sum()
        correct_adv_pred += (predicted_adv_labels == y_adv).sum()

    epoch_class_loss = running_class_loss / len(train_loader.dataset)
    epoch_adv_loss = running_adv_loss / len(train_loader.dataset)
    class_acc = correct_class_pred.float() / n
    adv_acc = correct_adv_pred.float() / n
        
    return model, epoch_class_loss, epoch_adv_loss, class_acc, adv_acc

In [13]:
def training_loop(model, criterion, optimizer, train_loader, valid_loader, epochs, device, adv_training, adv_training_prob, adv_val, logs, print_every=1):
    '''
    Function defining the entire training loop
    '''
    
    # set objects for storing metrics
    best_loss = 1e10
    train_class_losses = []
    valid_class_losses = []
    train_adv_losses = []
    valid_adv_losses = []
    start_epoch = logs['epochs']
 
    # Train model
    for epoch in range(start_epoch, epochs):

        # training
        if adv_training:
            model, optimizer, train_class_loss, train_adv_loss, train_class_acc, train_adv_acc = adv_train(train_loader, model, criterion, optimizer, device, adv_training_prob)
        else:
            model, optimizer, train_class_loss, train_adv_loss, train_class_acc, train_adv_acc = train(train_loader, model, criterion, optimizer, device)
        train_class_losses.append(train_class_loss)
        train_adv_losses.append(train_adv_loss)

        # validation
        if adv_val:
            model, adv_valid_class_loss, adv_valid_adv_loss, adv_valid_class_acc, adv_valid_adv_acc = adv_validate(valid_loader, model, criterion, device)
        with torch.no_grad():
            model, valid_class_loss, valid_adv_loss, valid_class_acc, valid_adv_acc = validate(valid_loader, model, criterion, device)
            valid_class_losses.append(valid_class_loss)
            valid_adv_losses.append(valid_adv_loss)

        if epoch % print_every == (print_every - 1):
                        
            logs['epochs'] = epoch + 1
            logs['Train class loss'].append(train_class_loss)
            logs['Valid class loss'].append(valid_class_loss)
            logs['Train adv loss'].append(train_adv_loss)
            logs['Valid adv loss'].append(valid_adv_loss)
            logs['Train class acc'].append(train_class_acc)
            logs['Valid class acc'].append(valid_class_acc)
            logs['Train adv acc'].append(train_adv_acc)
            logs['Valid adv acc'].append(valid_adv_acc)
            
            if adv_val:
                logs['ADV Valid class acc'].append(adv_valid_class_acc)
                logs['ADV Valid adv acc'].append(adv_valid_adv_acc)
                logs['ADV Valid class loss'].append(adv_valid_class_loss)
                logs['ADV Valid adv loss'].append(adv_valid_adv_loss)
                print(f'{datetime.now().time().replace(microsecond=0)} --- '
                      f'Epoch: {epoch}\t'
                      f'Train class acc: {100 * train_class_acc:.2f}\t'
                      f'Train class loss: {100 * train_class_loss:.2f}\t'
                      f'Train adv acc: {100 * train_adv_acc:.2f}\t'
                      f'Train adv loss: {100 * train_adv_loss:.2f}\t'
                      f'Valid class acc: {100 * valid_class_acc:.2f}\t'
                      f'Valid adv acc: {100 * valid_adv_acc:.2f}\t'
                      f'ADV Valid class acc: {100 * adv_valid_class_acc:.2f}\t'
                      f'ADV Valid adv acc: {100 * adv_valid_adv_acc:.2f}\t')
            else:
                print(f'{datetime.now().time().replace(microsecond=0)} --- '
                      f'Epoch: {epoch}\t'
                      f'Train class acc: {100 * train_class_acc:.2f}\t'
                      f'Train adv acc: {100 * train_adv_acc:.2f}\t'
                      f'Valid class acc: {100 * valid_class_acc:.2f}\t'
                      f'Valid adv acc: {100 * valid_adv_acc:.2f}\t')
            
            # save_model(model, optimizer, epoch, logs)

    # plot_losses(train_losses, valid_losses)
    
    return model, optimizer

In [14]:
def only_val_loop(model, criterion, valid_loader, epochs, device, adv_val, print_every=1):
    
    valid_class_losses = []
    valid_adv_losses = []

    for epoch in range(0, epochs):

        # validation
        if adv_val:
            model, adv_valid_class_loss, adv_valid_adv_loss, adv_valid_class_acc, adv_valid_adv_acc = adv_validate(valid_loader, model, criterion, device)
        with torch.no_grad():
            model, valid_class_loss, valid_adv_loss, valid_class_acc, valid_adv_acc = validate(valid_loader, model, criterion, device)
            valid_class_losses.append(valid_class_loss)
            valid_adv_losses.append(valid_adv_loss)

        if epoch % print_every == (print_every - 1):
                        
            if adv_val:
                print(f'{datetime.now().time().replace(microsecond=0)} --- '
                      f'Epoch: {epoch}\t'
                      f'Valid class acc: {100 * valid_class_acc:.2f}\t'
                      f'Valid adv acc: {100 * valid_adv_acc:.2f}\t'
                      f'ADV Valid class acc: {100 * adv_valid_class_acc:.2f}\t'
                      f'ADV Valid adv acc: {100 * adv_valid_adv_acc:.2f}\t')
            else:
                print(f'{datetime.now().time().replace(microsecond=0)} --- '
                      f'Epoch: {epoch}\t'
                      f'Valid class acc: {100 * valid_class_acc:.2f}\t'
                      f'Valid adv acc: {100 * valid_adv_acc:.2f}\t')
    
    return model, adv_valid_class_acc

In [15]:
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

In [16]:
N_EPOCHS = 10
LEARNING_RATE = 0.0005

adv_training = True
adv_training_prob = 0.5
adv_val = True

torch.manual_seed(RANDOM_SEED)

model = ClassificationDefense(n_adv_classes=2, n_classes=10).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.CrossEntropyLoss()

logs = {}
logs['epochs'] = 0
logs['Train class loss'] = []
logs['Valid class loss'] = []
logs['Train class acc'] = []
logs['Valid class acc'] = []
logs['ADV Valid class acc'] = []
logs['ADV Valid class loss'] = []
logs['Train adv loss'] = []
logs['Valid adv loss'] = []
logs['Train adv acc'] = []
logs['Valid adv acc'] = []
logs['ADV Valid adv acc'] = []
logs['ADV Valid adv loss'] = []

In [17]:
pgd_linf = PGD(loss_fn=torch.nn.CrossEntropyLoss(), num_steps = 10, step_size = 0.03, eps = 0.3, step_norm = 'inf', eps_norm = 'inf')
pgd_l2 = PGD(loss_fn=torch.nn.CrossEntropyLoss(), num_steps = 10, step_size = 0.3, eps = 3, step_norm = 2, eps_norm = 2)
fgsm = FGSM(loss_fn=torch.nn.CrossEntropyLoss(), fgsm_epsilon = 0.2)
cw = torchattacks.CW(model, c=10, kappa=0, steps=10, lr=0.5)
deepfool = torchattacks.DeepFool(model, steps=50, overshoot=0.2)

In [ ]:
model, optimizer = training_loop(model, criterion, optimizer, train_loader, valid_loader, 160, device, adv_training, adv_training_prob, adv_val, logs)

100%|██████████| 188/188 [00:00<00:00, 486.36it/s]


13:57:57 --- Epoch: 155	Train class acc: 96.03	Train class loss: 13.23	Train adv acc: 95.91	Train adv loss: 11.33	Valid class acc: 98.82	Valid adv acc: 98.34	ADV Valid class acc: 91.98	ADV Valid adv acc: 93.30	


 24%|██▍       | 180/750 [00:09<00:30, 18.57it/s]


KeyboardInterrupt: ignored

In [ ]:
# pickle.dump(model, open('/content/drive/MyDrive/Adversarial Machine Learning/Trained Model/MNIST_classification_defense_model-81%.pkl', 'wb'))

In [ ]:
pgd_linf.num_steps = 10
pgd_linf.eps = 0.2
pgd_linf.step_size = 0.02

In [19]:
if device == 'cpu':
    model = CPU_Unpickler(open('/content/drive/MyDrive/Adversarial Machine Learning/Trained Model/MNIST_classification_defense_model-81%.pkl', 'rb')).load()
else:
    model = pickle.load(open('/content/drive/MyDrive/Adversarial Machine Learning/Trained Model/MNIST_classification_defense_model-81%.pkl', 'rb'))

pgd_epsilon_list = list(np.linspace(0, 1, num=100, endpoint=True))
# fgsm_epsilon_list = range(0, 50, 1)
acc_dict = {}
logs = {}

for pgd_epsilon in pgd_epsilon_list:

    pgd_linf.eps = pgd_epsilon
    pgd_linf.step_size = pgd_linf.num_steps/pgd_epsilon

    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss()
    print('PGD attack with epsilon', pgd_linf.eps, 'result:')
    model, val_acc = only_val_loop(model, criterion, valid_loader, 1, device, adv_val = True)

    acc_dict[pgd_epsilon] = val_acc.cpu()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in double_scalars
  


PGD attack with epsilon 0.0 result:


100%|██████████| 188/188 [00:01<00:00, 158.09it/s]


14:22:35 --- Epoch: 0	Valid class acc: 98.80	Valid adv acc: 95.14	ADV Valid class acc: 98.80	ADV Valid adv acc: 4.86	
PGD attack with epsilon 0.010101010101010102 result:


100%|██████████| 188/188 [00:01<00:00, 154.72it/s]


...

15:28:16 --- Epoch: 0	Valid class acc: 98.80	Valid adv acc: 95.14	ADV Valid class acc: 10.81	ADV Valid adv acc: 99.91	
PGD attack with epsilon 0.98989898989899 result:


100%|██████████| 188/188 [00:01<00:00, 153.70it/s]


15:28:56 --- Epoch: 0	Valid class acc: 98.80	Valid adv acc: 95.14	ADV Valid class acc: 11.11	ADV Valid adv acc: 99.89	
PGD attack with epsilon 1.0 result:


100%|██████████| 188/188 [00:01<00:00, 155.99it/s]

15:29:37 --- Epoch: 0	Valid class acc: 98.80	Valid adv acc: 95.14	ADV Valid class acc: 11.32	ADV Valid adv acc: 99.90	
